In [ ]:
#drive.mount('/gdrive', force_remount=True)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11943450570399094868, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16185556992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6674932749774634881
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
import keras
keras.__version__

'2.6.0'

In [4]:
!pip install imgaug --upgrade

     |████████████████████████████████| 948 kB 2.7 MB/s 
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [5]:
# GPU 환경 설정
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 파일경로 설정
import json

# 데이터 보기
import pandas as pd
import numpy as np
from glob import glob

# 이미지데이터 로딩
import cv2
from tqdm import tqdm

# Others
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [6]:
!unzip -qq "/content/drive/MyDrive/Project/dacon/vision/new_images.zip"

In [7]:
'''Generator'''
from tensorflow.keras.utils import Sequence
from google.colab.patches import cv2_imshow
from imgaug import augmenters as iaa
from imgaug.augmentables.batches import UnnormalizedBatch

class Generator(Sequence):
    def __init__(self, src_path, input_shape, batch_size, augs=None, is_train=True):
        
        new_image_directory = src_path + '/new_images'
        new_train_image_directory = new_image_directory + '/train'

        action_information = pd.read_csv(src_path + '/action_information.csv')

        classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()
        
        new_train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1]))

        train_answer = []
        train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('_')[-1]))
        for train_image_directory in train_image_directories : 
            json_path = glob(train_image_directory + '/*.json')[0]
            js = json.load(open(json_path))
            action = js.get('action')
            train_answer.append(action)
        
        self.class_num = len(classes)
        self.is_train = is_train
        self.x_data = []
        self.y_data = []
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.augs = iaa.Sequential(augs)

        for new_train_image_directory, action in tqdm(zip(new_train_image_directories, train_answer), total = len(new_train_image_directories)) : 
            image_paths = sorted(glob(new_train_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
            image_len = len(image_paths)
            self.x_data +=image_paths
            
            for i in range(image_len):
                self.y_data.append(classes[action])

        self.on_epoch_end()
        print("Generator Initialized!!")
    
    def __len__(self):
        return round(len(self.x_data) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_x, batch_y = [], []
        batch_index = self.index[idx * self.batch_size:(idx+1)*self.batch_size]
        
        for batch_i in batch_index:
            batch_x.append(self.x_data[batch_i])
            batch_y.append(self.y_data[batch_i])
        
        out_x, out_y = self.data_gen(batch_x, batch_y)
        return out_x, out_y

    def on_epoch_end(self):
        self.index = np.arange(len(self.x_data))
        if self.is_train:
            np.random.shuffle(self.index)
    
    def data_gen(self, x ,y):
        input_x = np.zeros((self.batch_size, self.input_shape[0], self.input_shape[1], 3), dtype=np.float)
        input_y = np.zeros((self.batch_size, self.class_num), dtype=np.float)

        imgs = []
        for i in range(len(x)):
            img = cv2.imread(x[i])
            imgs.append(cv2.resize(img, (self.input_shape[0], self.input_shape[1])))
        
        batch_imgs = UnnormalizedBatch(images=imgs, data=y)
        batch_aug_imgs = list(self.augs.augment_batches(batches=batch_imgs))

        for i in range(len(x)):
            aug_img = batch_aug_imgs[0].images_aug[i]
            input_x[i]= aug_img.astype(np.float) / 255.0
            input_y[i] = y[i]
        return input_x, input_y
        


In [8]:
!ls /content/drive/MyDrive/Project/dacon/vision/

new_images.zip


In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def buildModel():
    baseModel = ResNet50V2(input_shape = (224,224,3), weights='imagenet', include_top=False, )
    baseModel.trainable = True
    model_in = Input(shape = (224,224,3))
    base_model = baseModel(model_in)
    head_model = GlobalAveragePooling2D()(base_model)
    head_model = Flatten(name="flatten")(head_model)
    head_model = Dense(256, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    head_model = Dense(32, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    head_model = Dense(8, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    model_out = Dense(6, activation="softmax")(head_model)

    model = Model(inputs=model_in, outputs=model_out)
    return model

In [11]:
import imgaug.augmenters as iaa

augs =[
       iaa.Fliplr(0.5),
       
        iaa.Sometimes(0.2,iaa.Sharpen()),
       
        iaa.SomeOf((1,2),[
            iaa.MultiplyAndAddToBrightness(),
            iaa.GammaContrast()
        ]),

        iaa.SomeOf((0,1), [
            iaa.Sometimes(0.7, iaa.AdditiveGaussianNoise()),
            iaa.Sometimes(0.7, iaa.GaussianBlur())
        ]),

        iaa.SomeOf((0,8),[
            iaa.ShearX(),
            iaa.ShearY(),
            iaa.ScaleX(),
            iaa.ScaleY(),
            iaa.TranslateX(),
            iaa.TranslateY(),
            iaa.Sometimes(0.3, iaa.Affine()),
            iaa.Sometimes(0.3, iaa.PerspectiveTransform()),
        ]),

        iaa.SomeOf((0,1),[
            iaa.Sometimes(0.6, iaa.Dropout()),
            iaa.Sometimes(0.6, iaa.CoarseDropout()),
            iaa.Sometimes(0.6, iaa.Cutout())
        ])
]

In [14]:
# Modeling
import tensorflow as tf
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

'''Train options'''
src_path = '/content/data'
batch_size = 16
input_shape = (224, 224)
epochs = 100
lr = 1e-4
model_name = "ResNet50V2"
filename = src_path+"/save_weights/%s_{epoch:05d}.h5"%(model_name)

callback = [ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, verbose=1),
            ModelCheckpoint(filename, monitor="loss", save_best_only=True, save_weights_only=True, verbose=1),
            EarlyStopping(monitor='loss', patience=5)]

In [16]:
'''Build Model'''
with tf.device('/device:GPU:0'):
    model = buildModel()

    model.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=lr, momentum=0.9), metrics=['accuracy'])


    '''Train'''
    train_gen = Generator(src_path, input_shape, batch_size, augs=None, is_train=True)
    model.fit_generator(train_gen, epochs=epochs, max_queue_size=50, workers=32, callbacks=callback)

100%|██████████| 142/142 [00:00<00:00, 1765.35it/s]


Generator Initialized!!
Epoch 1/100
1389/1389 [==============================] - 153s 95ms/step - loss: 1.5441 - accuracy: 0.3541

Epoch 00001: loss improved from inf to 1.54409, saving model to /content/data/save_weights/ResNet50V2_00001.h5
Epoch 2/100
1389/1389 [==============================] - 133s 96ms/step - loss: 0.8839 - accuracy: 0.6556

Epoch 00002: loss improved from 1.54409 to 0.88387, saving model to /content/data/save_weights/ResNet50V2_00002.h5
Epoch 3/100
1389/1389 [==============================] - 133s 95ms/step - loss: 0.4986 - accuracy: 0.8076

Epoch 00003: loss improved from 0.88387 to 0.49856, saving model to /content/data/save_weights/ResNet50V2_00003.h5
Epoch 4/100
1389/1389 [==============================] - 133s 96ms/step - loss: 0.3215 - accuracy: 0.8777

Epoch 00004: loss improved from 0.49856 to 0.32151, saving model to /content/data/save_weights/ResNet50V2_00004.h5
Epoch 5/100
1389/1389 [==============================] - 133s 96ms/step - loss: 0.2500 - acc

In [17]:
!nvidia-smi  

Thu Oct 14 02:23:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    34W / 250W |   8737MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
'''for test generator'''
src_path = '/content/data'
batch_size = 1
input_shape = (224, 224)
gen = Generator(src_path, input_shape, batch_size, is_train=True)
gen.__getitem__(0)

100%|██████████| 142/142 [00:00<00:00, 1793.60it/s]

Generator Initialized!!


(array([[[[0.21960784, 0.30196078, 0.27058824],
          [0.21960784, 0.30196078, 0.27058824],
          [0.21960784, 0.30196078, 0.27058824],
          ...,
          [0.38823529, 0.45490196, 0.43921569],
          [0.36078431, 0.42745098, 0.41176471],
          [0.3372549 , 0.40392157, 0.38823529]],
 
         [[0.16470588, 0.24705882, 0.21568627],
          [0.16470588, 0.24705882, 0.21568627],
          [0.16078431, 0.24313725, 0.21176471],
          ...,
          [0.40784314, 0.4745098 , 0.45882353],
          [0.41176471, 0.47843137, 0.4627451 ],
          [0.41176471, 0.47843137, 0.4627451 ]],
 
         [[0.12941176, 0.21176471, 0.18039216],
          [0.12941176, 0.21176471, 0.18039216],
          [0.13333333, 0.21568627, 0.18431373],
          ...,
          [0.21568627, 0.28235294, 0.26666667],
          [0.22352941, 0.29019608, 0.2745098 ],
          [0.23137255, 0.29803922, 0.28235294]],
 
         ...,
 
         [[0.53333333, 0.56078431, 0.61960784],
          [0.52941

# Prediction

In [20]:
src_path = '/content/data'

new_test_image_directory = src_path + '/new_images/test'
new_test_image_directories = sorted(glob(new_test_image_directory + '/*'), key = lambda x : int(x.split('file_')[-1]))
model = buildModel()
model.load_weights(src_path+"/save_weights/ResNet50V2_00052.h5")

predictions = []
for new_test_image_directory in tqdm(new_test_image_directories, total = len(new_test_image_directories)) :
    image_paths = sorted(glob(new_test_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
    image_len = len(image_paths)
    test_images  = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        img = img/255
        test_images.append(img)
    prediction = np.mean(model.predict(np.array(test_images)), axis = 0)
    predictions.append(prediction)

100%|██████████| 45/45 [00:31<00:00,  1.45it/s]


In [21]:
sample_submission = pd.read_csv(src_path + '/sample_submission.csv')
sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv('/content/drive/MyDrive/ResNet50V2_None_aug.csv', index=False)

,file_path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5
0,./test\file_142,0.019513,0.001004,0.830342,0.007520,7.962286e-03,0.133659
1,./test\file_143,0.001247,0.000008,0.000136,0.000178,4.978684e-04,0.997933
2,./test\file_144,0.002709,0.000071,0.000001,0.968129,3.760524e-07,0.029089
3,./test\file_145,0.029494,0.000153,0.772151,0.001168,8.717516e-03,0.188316
4,./test\file_146,0.998040,0.000638,0.000034,0.000267,6.505124e-04,0.000371


In [ ]:
# sample_submission = pd.read_csv(src_path + '/sample_submission.csv')
# sample_submission.iloc[:,1:] = predictions
# display(sample_submission.head())
# sample_submission.to_csv(src_path+'/Resnet50v2_Noneaug.csv', index=False)